# 训练篇

1. 选择nn.loss损失函数
2. 选择torch.optim优化算法
3. 设置超参数
4. 设置tensorboard进行可视化

In [6]:
import torch 
import torch.nn as nn # 包含loss和各种blocks, layers
from torch.utils.data import * # 包括Dataset和DataLoader
from tensorboardX import SummaryWriter 

from resnet import * 
from generate_dataset import * 

配置如下：
1. settings: 训练集地址和验证集地址
2. writer for tensorboard
3. prepare dataloader
4. load model
5. loss func 
6. optim 
7. let's go 

然后过程如下:
```python
for ep in range(EPOCHS):
    training ...
    validation ...
testing ...
```



In [7]:
torch.cuda.is_available()

True

In [8]:
torch.manual_seed(1)

In [9]:
def train(model_dir):
    # path for training and testing data
    train_data_path = "/share/mal/tmp/mal/train"
    test_data_path = "/share/mal/tmp/mal/test"
    
    # writer for debug
    writer = SummaryWriter(comment="malware_classification%resnet34")
    
    # prepare dataloader
    train_set = generate_dataset(train_data_path)
    test_set = generate_dataset(test_data_path)
    train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=64, shuffle=True)
    
    # load model
    model = ResNet(num_block=[3, 4, 6, 3], num_classes=9)
    model.cuda()
    model.train()
    
    # loss func
    criterion = nn.CrossEntropyLoss()
    
    # optim
    optim = torch.optim.Adam(model.parameters(), lr=1e-3)
    
    # let's go 
    EPOCH = 20
    step = 0
    best_acc = 0.0
    
    for ep in range(EPOCH):
        # training
        for idx, data in enumerate(train_loader):
            x, y = data
            y = torch.from_numpy(np.asarray(y, dtype=np.long))
            y-=1 # 训练集标签从1-9，映射到0-8
            x = x.cuda()
            y = y.cuda()
            
            y_pred = model(x)
            loss = criterion(y_pred, y)
            loss.backward()
            optim.step()
            optim.zero_grad()
            
            if step % 10 == 0:
                print ("epoch {} step {}: loss={}".format(ep, step, loss))
            writer.add_scalar("Loss", loss, step)
            step += 1
        
        # validation
        correct = 0
        total = 0
        model.eval()
        for idx, data in enumerate(test_loader):
            x, y = data
            y = torch.from_numpy(np.asarray(y, dtype=np.long))
            y-=1
            x = x.cuda()
            y = y.cuda()

            y_pred = model(x)
            prediction = torch.argmax(y_pred, 1)
            correct += (prediction == y).sum()
            total += len(y)
        
        acc = correct/total
        print ("epoch {}: acc={}".format(ep, acc))
        writer.add_scalar("acc", acc, ep)
        
        # save the epoch model and replace the best model
        model_path = os.path.join(model_dir, 'malware_classification%resnet34%{}.pth'.format(ep))
        torch.save(model, model_path)
        if acc > best_acc:
            best_model_path = os.path.join(model_dir, 'malware_classification%resnet34%best.pth')
            torch.save(model, best_model_path)
            best_acc = acc
            
    writer.close()

In [10]:
model_path = "/root/paperwithcode/第三周-训练篇/malware_dpcnn/trained_models/" 
train(model_path)

epoch 0 step 0: loss=2.2055020332336426
epoch 0 step 10: loss=1.5764442682266235
epoch 0 step 20: loss=1.2501105070114136
epoch 0 step 30: loss=1.1748971939086914
epoch 0 step 40: loss=0.9152734875679016
epoch 0 step 50: loss=0.7790653109550476
epoch 0 step 60: loss=0.790437638759613
epoch 0 step 70: loss=0.8054605722427368
epoch 0 step 80: loss=1.6085131168365479
epoch 0 step 90: loss=0.6463132500648499
epoch 0 step 100: loss=0.6345608234405518
epoch 0 step 110: loss=0.6703757643699646
epoch 0 step 120: loss=0.6758723855018616
epoch 0 step 130: loss=0.6089318990707397
epoch 0: acc=0.5085371136665344
epoch 1 step 140: loss=2.3388733863830566
epoch 1 step 150: loss=2.1149935722351074
epoch 1 step 160: loss=1.6574522256851196
epoch 1 step 170: loss=1.865393877029419
epoch 1 step 180: loss=1.8195676803588867
epoch 1 step 190: loss=1.8472682237625122
epoch 1 step 200: loss=1.6907707452774048
epoch 1 step 210: loss=1.6089190244674683
epoch 1 step 220: loss=1.7783348560333252
epoch 1 step 23

In [ ]:
# 保存最优模型字典
best_model_path = os.path.join(model_path, 'malware_classification%resnet34%best.pth')
state_dict_path = os.path.join(model_path, 'malware_classification%resnet34%best.pt')
model = ResNet(num_block=[3, 4, 6, 3])
best_model = torch.load(best_model_path)
torch.save(best_model.state_dict(), state_dict_path)